# README

The spatial analysis was implemented using an arcpy script `species_richness_ver_multiprocessing_colleen.py`. The central idea is to get a two-columns relationship between the OIDs of `species data` and that of protected areas/ICCA/merge (protected area input data, hereafter referred to as `PA`) spatially. Details of how this analysis was carried out can be found in the script and won't be repeated here.

The steps presented here concern non-spatial analysis of the resulting table from the spatial analysis. Notably the resulting table is joined by two lookup tables in order to bring in additional attributes from both `PA` and `species data`. It is these attributes that are used to group data, for example by red list categories, by country etc. By design, duplications of record exist and thus it is of great importance that **distinct** or **unique** should be used when counting.

# Load data and join lookup tables

In [186]:
# load all needed packages
import pandas as pd
import numpy as np

# result table species_oid - pa_oid
result = pd.read_csv("result.csv")

# look up attributes
rl_lp = pd.read_csv("rl_lookup.csv")
pa_lp = pd.read_csv("pa_lookup.csv")

In [187]:
rl_lp.columns

Index(['OBJECTID', 'id_no', 'binomial', 'presence', 'origin', 'seasonal',
       'compiler', 'year', 'citation', 'source', 'dist_comm', 'island',
       'subspecies', 'subpop', 'legend', 'tax_comm', 'kingdom_name',
       'phylum_name', 'class_name', 'order_name', 'family_name', 'genus_name',
       'friendly_name', 'code', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [188]:
temp_rl = pd.merge(result, rl_lp, left_on='OIDFC1', right_on='OBJECTID')

In [189]:
temp_rl = temp_rl.drop('OBJECTID', 1)

In [190]:
temp_rl.columns

Index(['OIDFC1', ' OIDFC2', 'id_no', 'binomial', 'presence', 'origin',
       'seasonal', 'compiler', 'year', 'citation', 'source', 'dist_comm',
       'island', 'subspecies', 'subpop', 'legend', 'tax_comm', 'kingdom_name',
       'phylum_name', 'class_name', 'order_name', 'family_name', 'genus_name',
       'friendly_name', 'code', 'Shape_Length', 'Shape_Area'],
      dtype='object')

** For what ever reason, `OIDFC2` has become ` OIDFC2` (with a preceding space)

In [191]:
full_result = pd.merge(temp_rl, pa_lp, left_on=' OIDFC2', right_on='OBJECTID')

In [192]:
full_result.columns

Index(['OIDFC1', ' OIDFC2', 'id_no', 'binomial', 'presence', 'origin',
       'seasonal', 'compiler', 'year', 'citation', 'source', 'dist_comm',
       'island', 'subspecies', 'subpop', 'legend', 'tax_comm', 'kingdom_name',
       'phylum_name', 'class_name', 'order_name', 'family_name', 'genus_name',
       'friendly_name', 'code', 'Shape_Length_x', 'Shape_Area_x', 'OBJECTID',
       'cat', 'iso3', 'Shape_Length_y', 'Shape_Area_y'],
      dtype='object')

In [194]:
# the total number of species involved (without POS filter)
full_result.id_no.unique().size

10695

In [193]:
# more concise df
concise_result = full_result[['id_no', 'binomial', 'class_name', 'code', 'cat', 'iso3']]

# Filters

Here I specify conditions for dividing the data into segments

In [ ]:
# == the below two conditions must always be met to ensure valid RL intepretation ===

# valid presence (1-2), origin (1-2) and seasonality (1-3)
pos_condition = full_result.presence.isin([1,2]) & full_result.origin.isin([1,2]) & full_result.seasonal.isin([1,2,3])

# exclude EX EW categories
rl_condition = full_result.code.str.upper().isin(['LC', 'NT', 'VU', 'EN', 'CR', 'DD'])


In [ ]:
# threatened RL species, VU, EN and CR
thr_condition = full_result.code.str.upper().isin(['VU', 'EN', 'CR'])

In [198]:
# species filters
mammal_condition = full_result.class_name.str.upper().isin(['MAMMALIA'])
bird_condition = full_result.class_name.str.upper().isin(['AVES'])
amphibian_condition = full_result.class_name.str.upper().isin(['AMPHIBIA'])
reptile_condition = full_result.class_name.str.upper().isin(['REPTILIA'])

In [199]:
# country and category conditions
aus = full_result.iso3.str.upper() == 'AUS'
bra = full_result.iso3.str.upper() == 'BRA'
nam = full_result.iso3.str.upper() == 'NAM'

comm = full_result.cat.str.upper() == 'COMM'
others = full_result.cat.str.upper() == 'OTHERS'
overlap = full_result.cat.str.upper() == 'OVERLAP'

In [200]:
# syntatic sugar if needed to look at detailed result
## base
base_filter = pos_condition & rl_condition

## handy shortcuts
filters = dict()
filters['aus'] = aus
filters['aus_comm'] = aus & comm
filters['aus_others'] = aus & others
filters['aus_overlap'] = aus & overlap
filters['bra'] = bra
filters['bra_comm'] = bra & comm
filters['bra_others'] = bra & others
filters['bra_overlap'] = bra & overlap
filters['nam'] = nam
filters['nam_comm'] = nam & comm
filters['nam_others'] = nam & others
filters['nam_overlap'] = nam & overlap

# (Checks)

Sanity checks on the result (can be safely ignored)

In [201]:
# the total number of species per PA merge
result.groupby(' OIDFC2').OIDFC1.nunique()

 OIDFC2
1     3920
2     4972
3     1758
4     4780
5     3839
6     5663
7     2451
8     3408
9     4444
10    1474
Name: OIDFC1, dtype: int64

In [202]:
concise_result.head(10)

,id_no,binomial,class_name,code,cat,iso3
0,569.0,Aethomys chrysophilus,MAMMALIA,LC,comm,NAM
1,569.0,Aethomys chrysophilus,MAMMALIA,LC,comm,NAM
2,811.0,Alcelaphus buselaphus,MAMMALIA,LC,comm,NAM
3,811.0,Alcelaphus buselaphus,MAMMALIA,RE,comm,NAM
4,811.0,Alcelaphus buselaphus,MAMMALIA,LC,comm,NAM
5,811.0,Alcelaphus buselaphus,MAMMALIA,RE,comm,NAM
6,2274.0,Atelerix frontalis,MAMMALIA,LC,comm,NAM
7,2428.0,Austroglanis sclateri,ACTINOPTERYGII,LC,comm,NAM
8,3755.0,Canis mesomelas,MAMMALIA,LC,comm,NAM
9,3847.0,Caracal caracal,MAMMALIA,LC,comm,NAM


In [203]:
# unique number of species in australia
concise_result[aus].id_no.nunique()

4571

In [204]:
# unique number of species in COMM areas
concise_result[comm].id_no.nunique()

8907

In [205]:
full_result.class_name.unique()

array(['MAMMALIA', 'ACTINOPTERYGII', 'AMPHIBIA', 'INSECTA', 'MALACOSTRACA',
       'LILIOPSIDA', 'MAGNOLIOPSIDA', 'BIVALVIA', 'GASTROPODA', 'REPTILIA',
       'POLYPODIOPSIDA', 'AVES', 'ISOETOPSIDA', 'DIPLOPODA',
       'CHONDRICHTHYES', 'HOLOTHUROIDEA', 'MYXINI', 'ANTHOZOA', 'HYDROZOA',
       'ARACHNIDA', 'CHAROPHYACEAE', 'AGARICOMYCETES', 'ENOPLA',
       'CEPHALASPIDOMORPHI'], dtype=object)

# Analysis

In [206]:
# group a give df by category and iso3
def group(df):
    """<df>, <query> -> <df>"""
    return df.groupby(['cat', 'iso3']).id_no.nunique().reset_index()

def group_by_country(df):
    return df.groupby('iso3').id_no.nunique()

One type of analysis is to look at the number of species per taxon and compare the richness within PA and ICCAs

In [207]:
mammal, amphibian, bird = [group(concise_result[base_filter&species_condition]) for species_condition in [mammal_condition, amphibian_condition, bird_condition]]

In [208]:
mammal_thr, amphibian_thr, bird_thr = [group(concise_result[base_filter&thr_condition&species_condition]) for species_condition in [mammal_condition, amphibian_condition, bird_condition]]

merge all the results into a single df

In [209]:
from functools import reduce

# use reduce function to merge any given number of dfs; here i used a tuple in order to get suffix information
# the first element is the df, the second a dummy value used in the reduce process
output = reduce(lambda left, right: (pd.merge(left, right, on=['iso3', 'cat'], how='left')),\
                  [mammal, amphibian, bird, mammal_thr, amphibian_thr, bird_thr])

output.columns = ['cat', 'iso3', 'n_mammals', 'n_amphibian', 'n_bird', 'thr_n_mammal', 'thr_n_amphibian', 'thr_n_bird']
output

,cat,iso3,n_mammals,n_amphibian,n_bird,thr_n_mammal,thr_n_amphibian,thr_n_bird
0,comm,AUS,256,142,658,35,22.0,43
1,comm,BRA,612,581,1656,72,8.0,120
2,comm,NAM,176,49,554,11,NaN,25
3,others,AUS,328,214,730,64,46.0,69
4,others,BRA,642,763,1718,83,27.0,154
5,others,NAM,210,52,578,18,1.0,35
6,overlap,AUS,222,116,585,25,17.0,26
7,overlap,BRA,560,499,1572,61,6.0,100
8,overlap,NAM,158,41,518,11,NaN,24


In [210]:
# fill NAs with 0 and export
output = output.fillna(0)
output.to_csv('output.csv')

all mammal, amphibians, birds

In [211]:
# separately for species per country, regardless of categories
output_all = reduce(lambda left, right: (pd.merge(left, right, on='iso3')), \
      [group_by_country(concise_result[base_filter&mammal_condition]).reset_index(), \
      group_by_country(concise_result[base_filter&amphibian_condition]).reset_index(), \
      group_by_country(concise_result[base_filter&bird_condition]).reset_index(), \
      group_by_country(concise_result[base_filter&thr_condition&mammal_condition]).reset_index(), \
      group_by_country(concise_result[base_filter&thr_condition&amphibian_condition]).reset_index(), \
      group_by_country(concise_result[base_filter&thr_condition&bird_condition]).reset_index()])

output_all.columns = ['iso3', 'n_mammals', 'n_amphibian', 'n_bird', 'thr_n_mammal', 'thr_n_amphibian', 'thr_n_bird']
output_all.to_csv('output_all.csv')

Richness alone tells only part of the story, as it ignores an important element that some of the species may be present in both PA and ICCAs. It is possible that species in ICCAs are a subset of the species in PAs, thus ICCA do not in theory provide additional benefits, from the point of view of species numbers. Or may be there are far fewer species in ICCAs but these species are not found in PAs. In this scenario, no matter how small the number is, they would contribute and complement that of PA. Maybe unnecesarily complicated and not needed...

In [212]:
# 1044 species in australia PA but not in COMM
np.setdiff1d(concise_result[pos_condition&rl_condition&aus&others].id_no, concise_result[pos_condition&rl_condition&aus&comm].id_no).size

1044

In [213]:
# 70 species in australia COMM but not in PA
np.setdiff1d(concise_result[pos_condition&rl_condition&aus&comm].id_no, concise_result[pos_condition&rl_condition&aus&others].id_no).size

70